# Recommendations
* The recommendations for a user will be output as a notebook in "../../data/recommendations/{username}/rec_egress/{username}.ipynb"

In [ ]:
import os
import shutil

import papermill as pm

In [ ]:
username = "Fro116" # PLACE THE USER YOU WANT RECOMMENDATIONS FOR HERE
source = "MAL" # PLACE THE USER'S LIST SOURCE HERE
task = "temporal_causal"
DAEMON_MODE = False

In [ ]:
assert username, "Must provide a username"
assert source in ["MAL", "AniList", "Kitsu", "Training"], "Must provide a source"

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
rec_dir = os.path.join(data_path, username)
papermill_path = os.path.join(rec_dir, "papermill")

In [ ]:
if DAEMON_MODE:
    print("Run the following commands to start julia daemons:")
    for i in range(6):
        print(f"julia -e 'using DaemonMode; serve(300{i}, true)' &")    
    julia_args = "-e 'using DaemonMode; runargs({port})'"    
else:
    julia_args = ""

In [ ]:
def run_julia_notebook(name, port, *args):
    os.system(f"julia {julia_args.format(port=port)} {name} {' '.join(args)}")

## RecIngress

In [ ]:
def run_ingress_notebook(name, output_name=None, **kwargs):
    outdir = os.path.join(papermill_path, "rec_ingress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    if output_name is None:
        output_name = name
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source, **kwargs),
    )

In [ ]:
def run_ingress_julia(name, *args):
    cwd = os.getcwd()
    os.chdir("../RecIngress")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", 3000, username, *args)
    finally:
        os.chdir(cwd)

In [ ]:
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
#TODO consolidate papermill startup costs

In [ ]:
run_ingress_notebook("FetchMediaLists", task=task)

In [ ]:
for medium in ["anime", "manga"]:
    run_ingress_notebook("ImportMediaLists", f"Import{medium}Lists", media=medium)
    run_ingress_notebook("ProcessMediaLists", f"Process{medium}Lists", media=medium)
    run_ingress_notebook("RelabelMediaIds", f"Relabel{medium}Ids", media=medium)
    run_ingress_notebook("GenerateMediaSplits", f"Generate{medium}Splits", media=medium)

In [ ]:
run_ingress_julia("CompressSplits")

## InferenceAlphas

In [ ]:
def run_inference_julia(name, port, *args):
    cwd = os.getcwd()
    os.chdir("../InferenceAlphas")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", port, username, *args)
    finally:
        os.chdir(cwd)

In [ ]:
run_inference_julia("Explicit/Explicit", 3001)

In [ ]:
run_inference_julia("Nondirectional/Nondirectional", 3002)

In [ ]:
run_inference_julia("Transformer/Transformer", 3003)

In [ ]:
run_inference_julia("Neural/Neural", 3004)

In [ ]:
run_inference_julia("Ensemble/Ensemble", 3005)

## RecEgress

In [ ]:
def run_egress_notebook(name, output_name, **kwargs):
    outdir = os.path.join(papermill_path, "rec_egress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)    
    pm.execute_notebook(
        f"{name}.ipynb",
        f"{outdir}/{output_name}.ipynb",
        parameters=dict(username=username, **kwargs),
    )

In [ ]:
if source == "Training":
    for medium in ["manga", "anime"]:
        run_egress_notebook(
            f"TrainingServingSkew",
            f"{medium.capitalize()}TrainingServingSkew",
            task=task,
            medium=medium,
        )
else:
    run_egress_notebook("Recommendations", username, source=source)

In [ ]:
print("Success!")